In [16]:
import numpy as np
import matplotlib.pyplot as plt
import math 
import pandas as pd

In [17]:
csv_file = "VASASHData.csv"

In [18]:
data = pd.read_csv(csv_file,encoding='latin1')

In [2]:
def allTargets(player_name,action):
    player_frame = frame[data['player']==player_name]
    pass_targets = dict()
    pass_count = 0
    for target in player_frame[player_frame['Action']==action]['Result']:
        if target in pass_targets:
            pass_targets[target] += 1
            pass_count += 1
        else:
            pass_targets[target] = 1
            pass_count += 1
    return pass_targets, pass_count


In [25]:
def actionFreq(player,action):
    action_dict,tot_cnt = allTargets(player,action)
    freq_dict = dict()
    for player in action_dict:
        freq_dict[player] = round(float(action_dict[player])/tot_cnt,5)
    return freq_dict

In [27]:
def dict2List(dict_):
    L = []
    for key in dict_:
        if not(key == 'NotCompleted'):
            L.append([dict_[key],key])
    L.sort()
    L_key = []
    L_ele = []
    for pair in L:
        L_key.append(pair[1])
        L_ele.append(pair[0])
    return L_key,L_ele

In [31]:
def playerChemistry(player_A, player_B, dict_A, dict_B):
    chem = 0
    if player_A in dict_B:
        chem = chem + dict_B[player_A]
    if player_B in dict_A:
        chem = chem + dict_A[player_B]
    return (chem/2)

In [34]:
def getPlayers(team):
    global player_nums
    players = dict()
    for p in frame[data['team']==team].player:
        if p in players:
            players[p] += 1
        else:
            players[p] = 1
    return players

In [36]:
def teamChemistry(team):
    players = getPlayers(team)
    chem_dict = dict()
    for player_A in players:
        dict_A = actionFreq(player_A,'pass')
        for player_B in players:
            if not(player_A == player_B):
                dict_B = actionFreq(player_B,'pass')
                chem = playerChemistry(player_A,player_B,dict_A,dict_B)
                if not((player_A,player_B) in chem_dict) and not((player_B,player_A) in chem_dict):
                    chem_dict[(player_A,player_B)] = [chem]
                elif ((player_A,player_B) in chem_dict) and not((player_B,player_A) in chem_dict):
                    chem_dict[(player_A,player_B)].append(chem)
    return chem_dict

In [37]:
def maxTeamChemistry(team):
    players = getPlayers(team)
    max_chem = 0
    max_A = []
    max_B = []
    for player_A in players:
        dict_A = actionFreq(player_A,'pass')
        for player_B in players:
            if not(player_A == player_B):
                dict_B = actionFreq(player_B,'pass')
                chem = playerChemistry(player_A,player_B,dict_A,dict_B)
                if chem > max_chem:
                    max_chem = chem
                    max_A = [player_A]
                    max_B = [player_B]
                elif chem == max_chem:
                    if not(player_A in max_B) or not(player_B in max_A):
                        max_A.append(player_A)
                        max_B.append(player_B)
    return(max_chem,max_A,max_B)

In [ ]:
def getStarters(team):
    players = getPlayers(team)
    n = 0
    starters = []
    for player in players:
        if n <= 11:
            starters.append([players[player],player])
            n += 1
        else:
            if players[player] > starters[0][0]:
                starters[0] = [players[player],player]
        starters.sort()
    return starters

In [39]:
def average_position(match, player):
    player_locX = data[data['Match']==match].groupby(['player'])['LocX'].apply(list)[0]
    player_locY = data[data['Match']==match].groupby(['player'])['LocY'].apply(list)[1]
    return np.average(player_locX[player]),np.average(player_locY[player])

In [58]:
def teamChemistryPlot(team,colours,match):
    global player_nums
    team_chem = teamChemistry(team)
    s_temp = getStarters(team)
    starters = dict()
    for starter in s_temp:
        starters[starter[1]] = starter[0]
    img = plt.imread("soccer_pitch.jpg")
    fig, ax = plt.subplots(figsize=(100,100))
    ax.imshow(img,extent=[0,100,0,100])
    plt.axis('off')
    player_loc = dict()
    for player in starters:
        print(player)
        y = data[data['team']==team][data['player']==player].mean()['LocX'].mean()
        x = data[data['team']==team][data['player']==player].mean()['LocY'].mean()
        if (x != 'nan') and (y != 'nan'):
            player_loc[player] = (x,y)
            if player in player_nums[team]:
                plt.annotate(player_nums[team][player], xy=(x,y))
            else:
                print('*** Fix Above ***')
            fig = plt.gcf()
            ax = fig.gca()
            #ax.annotate(player, xy=(x,y), xytext=(x,y))
            circle = plt.Circle((x,y), 2, color = colours[0], label=player)
            circle = plt.Circle((x,y), 1.75, color = colours[1], label=player)
            ax.add_artist(circle)
    for pair in team_chem:
        if (pair[0] in starters) and (pair[1] in starters):
            x1 = player_loc[pair[0]][0]
            y1 = player_loc[pair[0]][1]
            x2 = player_loc[pair[1]][0]
            y2 = player_loc[pair[1]][1]
            ax.plot((x1,x2),(y1,y2),linewidth=team_chem[pair][0]*300,c=colours[0])
    plt.show()

In [59]:
team_and_colours = {
 'Atlanta United':['#80000A', '#221F1F', '#A19060', '#817144'],
 'Chicago Fire':['#121F48', '#B3272D', '#909191'],
 'Colorado Rapids':['#862633', '#8BB8E8', '#8D9093'],
 'Columbus Crew':['#FEF200', '#231F20'],
 'DC United':['#EE1A39', '#231F20'],
 'FC Dallas':['#D11241', '#003E7E', '#D1D5D8'],
 'Houston Dynamo':['#F4911E', '#92C3F1', '#231F20'],
 'LA Galaxy':['#0065A4', '#00245D', '#C4C7DC', '#FFD200', '#EEB111', '#8C8B8C', '#B3B2B3', '#231F20'],
 'Minnesota United':['#585958', '#D8DAD9', '#231F20', '#DF2426', '#9BCDE4'],
 'Montreal Impact':['#2B63AD', '#28477D', '#373536', '#929397', '#BBC3C6'],
 'New England Revolution':['#E51938', '#002B5C'],
 'New York City FC':['#6CACE4', '#041E42', '#F15524'],
 'New York Red Bulls':['#E31351', '#002F65', '#FFC425'],
 'Orlando City SC':['#61259E', '#FFE293'],
 'Philadelphia Union':['#002D55', '#5090CD', '#B38707', '#B49759'],
 'Portland Timbers':['#EAE827', '#004812'],
 'Real Salt Lake':['#B30838', '#013A81', '#F5E700'],
 'San Jose Earthquakes':['#1F1F1F', '#30457A', '#A42A35'],
 'Seattle Sounders FC':['#236192', '#658D1B', '#1D252D'],
 'Sporting Kansas City':['#93B1D7', '#002A5C', '#A0A1A5'],
 'Toronto FC':['#AB1E2D', '#3F4743', '#A3AAAD'],
 'Vancouver Whitecaps FC':['#04265C', '#94C2E4', '#84868C']
}


In [64]:
def plotTeam(team_name):
    global team_and_colours
    colours = team_and_colours[team]
    teamChemistryPlot(team,colours,teams[team])